In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
import pickle

In [39]:
df= pd.read_excel("coffee_survey.xlsx")

In [40]:
#BASIC CHECKS :
pd.set_option('display.max_columns', None)
df.head()


submission_id              age cups where_drink  \
0        gMR29l  18-24 years old  NaN         NaN   
1        BkPN0e  25-34 years old  NaN         NaN   
2        W5G8jj  25-34 years old  NaN         NaN   
3        4xWgGr  35-44 years old  NaN         NaN   
4        QD27Q8  25-34 years old  NaN         NaN   

                                          brew brew_other purchase  \
0                                          NaN        NaN      NaN   
1  Pod/capsule machine (e.g. Keurig/Nespresso)        NaN      NaN   
2                          Bean-to-cup machine        NaN      NaN   
3     Coffee brewing machine (e.g. Mr. Coffee)        NaN      NaN   
4                                    Pour over        NaN      NaN   

  purchase_other             favorite favorite_specify  \
0            NaN  Regular drip coffee              NaN   
1            NaN          Iced coffee              NaN   
2            NaN  Regular drip coffee              NaN   
3            NaN          Iced coffee              NaN   
4            NaN                Latte              NaN   

                             additions additions_other dairy sweetener  \
0                      No - just black             NaN   NaN       NaN   
1  Sugar or sweetener, No - just black             NaN   NaN       NaN   
2                      No - just black             NaN   NaN       NaN   
3            No - just black, Cinnamon             NaN   NaN       NaN   
4                      No - just black             NaN   NaN       NaN   

     style strength roast_level caffeine  expertise  coffee_a_bitterness  \
0  Complex      NaN         NaN      NaN        NaN                  NaN   
1    Light      NaN         NaN      NaN        NaN                  NaN   
2  Complex      NaN         NaN      NaN        NaN                  NaN   
3  Complex      NaN         NaN      NaN        NaN                  NaN   
4    Sweet      NaN         NaN      NaN        NaN                  NaN   

   coffee_a_acidity  coffee_a_personal_preference coffee_a_notes  \
0               NaN                           NaN            NaN   
1               NaN                           NaN            NaN   
2               NaN                           NaN            NaN   
3               NaN                           NaN            NaN   
4               NaN                           NaN            NaN   

   coffee_b_bitterness  coffee_b_acidity  coffee_b_personal_preference  \
0                  NaN               NaN                           NaN   
1                  NaN               NaN                           NaN   
2                  NaN               NaN                           NaN   
3                  NaN               NaN                           NaN   
4                  NaN               NaN                           NaN   

  coffee_b_notes  coffee_c_bitterness  coffee_c_acidity  \
0            NaN                  NaN               NaN   
1            NaN                  NaN               NaN   
2            NaN                  NaN               NaN   
3            NaN                  NaN               NaN   
4            NaN                  NaN               NaN   

   coffee_c_personal_preference coffee_c_notes  coffee_d_bitterness  \
0                           NaN            NaN                  NaN   
1                           NaN            NaN                  NaN   
2                           NaN            NaN                  NaN   
3                           NaN            NaN                  NaN   
4                           NaN            NaN                  NaN   

   coffee_d_acidity  coffee_d_personal_preference coffee_d_notes prefer_abc  \
0               NaN                           NaN            NaN        NaN   
1               NaN                           NaN            NaN        NaN   
2               NaN                           NaN            NaN        NaN   
3               NaN                           NaN          

In [41]:
#Columns that are more than 20% null:
l= list()
for col in df.columns:
    p= (df[col].isnull().sum()*100)/len(df)
    if p>20:
      l.append(col)
      print(col," :",p,"%")

#These columns are be dropped.

brew_other  : 83.22612568035626 %
purchase  : 82.43443839683326 %
purchase_other  : 99.23305294408709 %
favorite_specify  : 97.17961405244928 %
additions_other  : 98.8124690747155 %
dairy  : 58.287976249381494 %
sweetener  : 87.33300346363187 %
coffee_a_notes  : 36.219693221177636 %
coffee_b_notes  : 39.28748144482929 %
coffee_c_notes  : 41.06877783275606 %
coffee_d_notes  : 35.97229094507669 %
why_drink_other  : 95.8683819891143 %
gender_specify  : 99.70311726867887 %
ethnicity_race_specify  : 97.40227610094013 %
number_children  : 78.82236516575952 %


In [42]:
df.drop(l,axis=1,inplace= True)
df.drop("submission_id",axis=1,inplace= True)
df.head()
#Drop submission_id , consist all unique values.

age cups where_drink  \
0  18-24 years old  NaN         NaN   
1  25-34 years old  NaN         NaN   
2  25-34 years old  NaN         NaN   
3  35-44 years old  NaN         NaN   
4  25-34 years old  NaN         NaN   

                                          brew             favorite  \
0                                          NaN  Regular drip coffee   
1  Pod/capsule machine (e.g. Keurig/Nespresso)          Iced coffee   
2                          Bean-to-cup machine  Regular drip coffee   
3     Coffee brewing machine (e.g. Mr. Coffee)          Iced coffee   
4                                    Pour over                Latte   

                             additions    style strength roast_level caffeine  \
0                      No - just black  Complex      NaN         NaN      NaN   
1  Sugar or sweetener, No - just black    Light      NaN         NaN      NaN   
2                      No - just black  Complex      NaN         NaN      NaN   
3            No - just black, Cinnamon  Complex      NaN         NaN      NaN   
4                      No - just black    Sweet      NaN         NaN      NaN   

   expertise  coffee_a_bitterness  coffee_a_acidity  \
0        NaN                  NaN               NaN   
1        NaN                  NaN               NaN   
2        NaN                  NaN               NaN   
3        NaN                  NaN               NaN   
4        NaN                  NaN               NaN   

   coffee_a_personal_preference  coffee_b_bitterness  coffee_b_acidity  \
0                           NaN                  NaN               NaN   
1                           NaN                  NaN               NaN   
2                           NaN                  NaN               NaN   
3                           NaN                  NaN               NaN   
4                           NaN                  NaN               NaN   

   coffee_b_personal_preference  coffee_c_bitterness  coffee_c_acidity  \
0                           NaN                  NaN               NaN   
1                           NaN                  NaN               NaN   
2                           NaN                  NaN               NaN   
3                           NaN                  NaN               NaN   
4                           NaN                  NaN               NaN   

   coffee_c_personal_preference  coffee_d_bitterness  coffee_d_acidity  \
0                           NaN                  NaN               NaN   
1                           NaN                  NaN               NaN   
2                           NaN                  NaN               NaN   
3                           NaN                  NaN               NaN   
4                           NaN                  NaN               NaN   

   coffee_d_personal_preference prefer_abc prefer_ad prefer_overall  wfh  \
0                           NaN        NaN       NaN            NaN  NaN   
1                           NaN        NaN       NaN            NaN  NaN   
2                           NaN        NaN       NaN            NaN  NaN   
3                           NaN        NaN       NaN            NaN  NaN   
4                           NaN        NaN       NaN            NaN  NaN   

  total_spend why_drink taste know_source most_paid most_willing value_cafe  \
0         NaN       NaN   NaN         NaN       NaN          NaN        NaN   
1         NaN       NaN   NaN         NaN       NaN          NaN        NaN   
2         NaN       NaN   NaN         NaN       NaN          NaN        NaN   
3         NaN       NaN   NaN         NaN       NaN          NaN        NaN   
4         NaN       NaN   NaN         NaN       NaN          NaN        NaN   

  spent_equipment value_equipment gender education_level ethnicity_race  \
0             NaN             NaN    NaN             NaN            NaN   
1             NaN             NaN    NaN             NaN            NaN   
2             NaN             NaN    NaN   

In [43]:
df.info()
#rows = 4042
#cols = 41

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4042 entries, 0 to 4041
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           4011 non-null   object 
 1   cups                          3949 non-null   object 
 2   where_drink                   3972 non-null   object 
 3   brew                          3657 non-null   object 
 4   favorite                      3980 non-null   object 
 5   additions                     3959 non-null   object 
 6   style                         3958 non-null   object 
 7   strength                      3916 non-null   object 
 8   roast_level                   3940 non-null   object 
 9   caffeine                      3917 non-null   object 
 10  expertise                     3938 non-null   float64
 11  coffee_a_bitterness           3798 non-null   float64
 12  coffee_a_acidity              3779 non-null   float64
 13  cof

In [44]:
df.describe()

expertise  coffee_a_bitterness  coffee_a_acidity  \
count  3938.000000          3798.000000       3779.000000   
mean      5.693499             2.141127          3.634824   
std       1.948867             0.947163          0.982113   
min       1.000000             1.000000          1.000000   
25%       5.000000             1.000000          3.000000   
50%       6.000000             2.000000          4.000000   
75%       7.000000             3.000000          4.000000   
max      10.000000             5.000000          5.000000   

       coffee_a_personal_preference  coffee_b_bitterness  coffee_b_acidity  \
count                   3789.000000          3780.000000       3767.000000   
mean                       3.310900             3.013228          2.223786   
std                        1.185953             0.992875          0.865389   
min                        1.000000             1.000000          1.000000   
25%                        2.000000             2.000000          2.000000   
50%                        3.000000             3.000000          2.000000   
75%                        4.000000             4.000000          3.000000   
max                        5.000000             5.000000          5.000000   

       coffee_b_personal_preference  coffee_c_bitterness  coffee_c_acidity  \
count                   3773.000000          3764.000000       3751.000000   
mean                       3.068646             3.071998          2.366836   
std                        1.113546             0.999267          0.921048   
min                        1.000000             1.000000          1.000000   
25%                        2.000000             2.000000          2.000000   
50%                        3.000000             3.000000          2.000000   
75%                        4.000000             4.000000          3.000000   
max                        5.000000             5.000000          5.000000   

       coffee_c_personal_preference  coffee_d_bitterness  coffee_d_acidity  \
count                   3766.000000          3767.000000       3765.000000   
mean                       3.064790             2.162729          3.858167   
std                        1.128431             1.081546          1.007973   
min                        1.000000             1.000000          1.000000   
25%                        2.000000             1.000000          3.000000   
50%                        3.000000             2.000000          4.000000   
75%                        4.000000             3.000000          5.000000   
max                        5.000000             5.000000          5.000000   

       coffee_d_personal_preference  
count                   3764.000000  
mean                       3.375930  
std                        1.452504  
min                        1.000000  
25%                        2.000000  
50%                        4.000000  
75%                        5.000000  
max                        5.000000

In [45]:
df.describe(include="O")

age  cups where_drink       brew  favorite  \
count              4011  3949        3972       3657      3980   
unique                7     6          65        449        12   
top     25-34 years old     2     At home  Pour over  Pourover   
freq               1986  1663        1625        575      1084   

              additions   style         strength roast_level       caffeine  \
count              3959    3958             3916        3940           3917   
unique               53      12                5           7              3   
top     No - just black  Fruity  Somewhat strong       Light  Full caffeine   
freq               2154     953             1791        1778           3576   

       prefer_abc prefer_ad prefer_overall                         wfh  \
count        3772      3761           3770                        3524   
unique          3         2              4                           3   
top      Coffee A  Coffee D       Coffee D  I primarily work from home   
freq         1682      2015           1385                        1592   

       total_spend       why_drink taste know_source most_paid most_willing  \
count         3511            3568  3563        3559      3527         3510   
unique           6              84     2           2         8            8   
top        $20-$40  It tastes good   Yes         Yes     $6-$8       $8-$10   
freq          1293             693  3460        2720      1095          880   

       value_cafe   spent_equipment value_equipment gender    education_level  \
count        3500              3506            3494   3523               3438   
unique          2                 7               2      5                  6   
top           Yes  More than $1,000             Yes   Male  Bachelor's degree   
freq         2048               780            3318   2524               1759   

         ethnicity_race   employment_status political_affiliation  
count              3418                3419                  3289  
unique                6                   6                     4  
top     White/Caucasian  Employed full-time              Democrat  
freq               2626                2708                  1768

In [46]:
df["why_drink"].unique()
#why_drink is not required, same thing written differently. Too messy.

array([nan, 'It tastes good', 'It tastes good, I need the caffeine',
       'It tastes good, I need the caffeine, I need the ritual',
       'I need the ritual, It tastes good',
       'It tastes good, I need the ritual',
       'I need the ritual, It tastes good, I need the caffeine',
       'I need the ritual, I need the caffeine, It tastes good',
       'It tastes good, I need the ritual, I need the caffeine',
       'I need the ritual',
       'It tastes good, I need the ritual, I need the caffeine, It makes me go to the bathroom',
       'It tastes good, I need the ritual, It makes me go to the bathroom',
       'I need the caffeine, It tastes good',
       'I need the caffeine, I need the ritual, It makes me go to the bathroom, It tastes good',
       'I need the ritual, I need the caffeine, It makes me go to the bathroom, It tastes good',
       'It tastes good, I need the caffeine, I need the ritual, It makes me go to the bathroom',
       'I need the caffeine',
       'I need 

In [47]:
df["brew"].unique()

array([nan, 'Pod/capsule machine (e.g. Keurig/Nespresso)',
       'Bean-to-cup machine', 'Coffee brewing machine (e.g. Mr. Coffee)',
       'Pour over',
       'Pod/capsule machine (e.g. Keurig/Nespresso), Espresso, French press',
       'Pour over, French press, Espresso, Instant coffee, Pod/capsule machine (e.g. Keurig/Nespresso)',
       'Pour over, Coffee brewing machine (e.g. Mr. Coffee), Espresso, Bean-to-cup machine',
       'Pour over, Espresso, Instant coffee',
       'Pour over, Coffee brewing machine (e.g. Mr. Coffee)', 'Other',
       'Espresso', 'Coffee extract (e.g. Cometeer)',
       'Pod/capsule machine (e.g. Keurig/Nespresso), French press',
       'Pour over, Espresso', 'Espresso, Cold brew, Pour over',
       'French press, Cold brew', 'Pour over, French press, Espresso',
       'Pour over, Cold brew, Other',
       'French press, Pod/capsule machine (e.g. Keurig/Nespresso), Other',
       'Pour over, French press, Espresso, Other, Cold brew, Coffee brewing machine (

In [48]:
df['additions'].unique()

array(['No - just black', 'Sugar or sweetener, No - just black',
       'No - just black, Cinnamon', 'Half & half', nan,
       'Milk, dairy alternative, or coffee creamer, Sugar or sweetener, Flavor syrup',
       'No - just black, Milk, dairy alternative, or coffee creamer',
       'Milk, dairy alternative, or coffee creamer, Sugar or sweetener',
       'Milk, dairy alternative, or coffee creamer',
       'Flavor syrup, Milk, dairy alternative, or coffee creamer',
       'No - just black, Milk, dairy alternative, or coffee creamer, Sugar or sweetener',
       'Milk, dairy alternative, or coffee creamer, Sugar or sweetener, No - just black',
       'Sugar or sweetener', 'No - just black, Other',
       'Milk, dairy alternative, or coffee creamer, No - just black',
       'Sugar or sweetener, Milk, dairy alternative, or coffee creamer, No - just black',
       'Other',
       'No - just black, Sugar or sweetener, Milk, dairy alternative, or coffee creamer',
       'Milk, dairy alternat

In [49]:
df['age'].unique()
#ordinal encoding required

array(['18-24 years old', '25-34 years old', '35-44 years old',
       '55-64 years old', nan, '<18 years old', '>65 years old',
       '45-54 years old'], dtype=object)

In [50]:
df['most_willing'].unique()
#ordinal encoding required

array([nan, '$8-$10', 'More than $20', '$15-$20', '$4-$6', '$6-$8',
       '$10-$15', '$2-$4', 'Less than $2'], dtype=object)

In [51]:
df["expertise"].unique()

array([nan,  7., 10.,  6.,  4.,  9.,  8.,  5.,  2.,  1.,  3.])

In [52]:
df["gender"].unique()

array([nan, 'Other (please specify)', 'Female', 'Male', 'Non-binary',
       'Prefer not to say'], dtype=object)

In [53]:
df["cups"].unique()

array([nan, 'Less than 1', 2, 1, 3, 'More than 4', 4], dtype=object)

In [54]:
df['favorite'].unique()

array(['Regular drip coffee', 'Iced coffee', 'Latte', 'Pourover', nan,
       'Other', 'Cortado', 'Cappuccino', 'Espresso', 'Cold brew',
       'Americano', 'Mocha', 'Blended drink (e.g. Frappuccino)'],
      dtype=object)

Milk-Heavy (Low Intensity): Drinks like Lattes or Mochas dilute the coffee. Users need Dark Roasts (Coffee C) to "punch through" the milk.

Water-Diluted (Medium Intensity): Drip or Americanos prioritize "sessionability." Users typically prefer Balanced Roasts (Coffee B) for easy drinking.

Concentrated (High Intensity): Espresso and Cortados focus on texture and body. Users seek Medium to Dark profiles with heavy mouthfeel.

Pure/Filter (High Clarity): Pourovers have zero dilution. These users have the highest acidity tolerance and prefer Light/Fruity Roasts (Coffee A or D).

In [55]:
df["taste"].value_counts()
#Since this is surveyed on coffee lovers, majority said yes, hence not useful for our model.

taste
Yes    3460
No      103
Name: count, dtype: int64

In [56]:
df["strength"].unique()

array([nan, 'Somewhat light', 'Somewhat strong', 'Medium', 'Very strong',
       'Weak'], dtype=object)

In [57]:
df['caffeine'].unique()

array([nan, 'Full caffeine', 'Half caff', 'Decaf'], dtype=object)

In [58]:
df.columns

Index(['age', 'cups', 'where_drink', 'brew', 'favorite', 'additions', 'style',
       'strength', 'roast_level', 'caffeine', 'expertise',
       'coffee_a_bitterness', 'coffee_a_acidity',
       'coffee_a_personal_preference', 'coffee_b_bitterness',
       'coffee_b_acidity', 'coffee_b_personal_preference',
       'coffee_c_bitterness', 'coffee_c_acidity',
       'coffee_c_personal_preference', 'coffee_d_bitterness',
       'coffee_d_acidity', 'coffee_d_personal_preference', 'prefer_abc',
       'prefer_ad', 'prefer_overall', 'wfh', 'total_spend', 'why_drink',
       'taste', 'know_source', 'most_paid', 'most_willing', 'value_cafe',
       'spent_equipment', 'value_equipment', 'gender', 'education_level',
       'ethnicity_race', 'employment_status', 'political_affiliation'],
      dtype='object')

In [59]:
#required columns :
req= ['age', 'cups', 'brew', 'favorite', 'additions', 'strength','caffeine', 'expertise','most_willing',
      'gender', 'prefer_overall']
data = df[req]
data.head()

age cups                                         brew  \
0  18-24 years old  NaN                                          NaN   
1  25-34 years old  NaN  Pod/capsule machine (e.g. Keurig/Nespresso)   
2  25-34 years old  NaN                          Bean-to-cup machine   
3  35-44 years old  NaN     Coffee brewing machine (e.g. Mr. Coffee)   
4  25-34 years old  NaN                                    Pour over   

              favorite                            additions strength caffeine  \
0  Regular drip coffee                      No - just black      NaN      NaN   
1          Iced coffee  Sugar or sweetener, No - just black      NaN      NaN   
2  Regular drip coffee                      No - just black      NaN      NaN   
3          Iced coffee            No - just black, Cinnamon      NaN      NaN   
4                Latte                      No - just black      NaN      NaN   

   expertise most_willing gender prefer_overall  
0        NaN          NaN    NaN            NaN  
1        NaN          NaN    NaN            NaN  
2        NaN          NaN    NaN            NaN  
3        NaN          NaN    NaN            NaN  
4        NaN          NaN    NaN            NaN

In [60]:
def cups_clean(x):
  print("DEBUG: Running cups_clean")
    # 1. Input Fix: Handle the DataFrame from the Pipeline
  if isinstance(x, pd.DataFrame):
    x = x.iloc[:, 0]

  cups_map = {
        'less than 1': 0.5,
        '1': 1.0,
        '2': 2.0,
        '3': 3.0,
        '4': 4.0,
        'more than 4': 5.0
  }

  y = x.map(cups_map).fillna(2.0)

  #Return as a 2D array for the next step
  return y.values.reshape(-1, 1)

cups_transform = FunctionTransformer(cups_clean)

In [61]:
def strength_clean(x):
  if isinstance(x, pd.DataFrame):
    x = x.iloc[:, 0]
  strength_map = {
      'Somewhat light' : 1,
      'Weak'           : 1,
      'Medium'         : 2,
      'Somewhat strong': 3,
      'Very strong'    : 3
  }
  y = x.map(strength_map).fillna(2)

  # 3. Return as a 2D array (required by Scikit-Learn)
  return y.values.reshape(-1, 1)

strength_transform = FunctionTransformer(strength_clean)

In [62]:
def caffeine_clean(x):

  # 1. Input Fix: Ensure we have a 2D format for the encoder
  if isinstance(x, pd.Series):
    x = x.to_frame()
  elif not isinstance(x, pd.DataFrame):
     # If it's a numpy array, make it 2D
    x = x.reshape(-1, 1)
  #Decaf : 1, Half caff : 2 , Full caffeine : 3
  caffeine_order= ['Decaf','Half caff','Full caffeine']
  enc = OrdinalEncoder(categories=[caffeine_order],
                         handle_unknown='use_encoded_value',
                         unknown_value=-1)
  y = enc.fit_transform(pd.DataFrame(x).fillna('Full caffeine'))
  return y

caffeine_transform = FunctionTransformer(caffeine_clean)


In [63]:
def expertise_clean(x):
  if isinstance(x, pd.DataFrame):
    x = x.iloc[:, 0]
  # 'coerce' turns text like "Other" into NaN (empty values)
  y = pd.to_numeric(x, errors= 'coerce').fillna(5)
  return y.values.reshape(-1, 1)

expertise_transform = FunctionTransformer(expertise_clean)

In [64]:
def gender_clean(x):
  if isinstance(x, pd.DataFrame):
    x = x.iloc[:, 0]

  gender_map = {
    'Male': 0,
    'Female': 1,
    'Non-binary': 2,
    'Prefer not to say': 2,
    'Other': 2
  }
  y = x.map(gender_map).fillna(2)

  return y.values.reshape(-1, 1)

gender_transform = FunctionTransformer(gender_clean)


In [65]:
def map_willingness(value):

  value = str(value).lower()
  if '$0-$2' in value or '$2-$4' in value:
    return 1 # Budget/Traditional
  elif '$4-$6' in value or '$6-$8' in value:
    return 2 # Mid-range/Specialty
  else:
    return 3 # Premium/Experimental

def most_willing_clean(x):
  #FIX THE INPUT (from 2D table to 1D list)
  if isinstance(x, pd.DataFrame):
    x = x.iloc[:, 0]

  y = x.apply(map_willingness)
  return y.values.reshape(-1, 1)

willing_transform = FunctionTransformer(most_willing_clean)

In [66]:
def brew_clean(x):

    # Ensure input is a DataFrame
  temp_df = pd.DataFrame(x)
  col_name = temp_df.columns[0]

  methods = ['Pour over', 'Espresso', 'French press', 'Drip coffee',
               'AeroPress', 'Pod/capsule', 'Instant coffee']

  new_df = pd.DataFrame(index=temp_df.index) # Keep the index for alignment

  for method in methods:

    new_name = f"brew_{method.lower().replace(' ', '_')}"
    # Use .str.contains to find the method anywhere in the string
    new_df[new_name] = temp_df[col_name].str.contains(method, case=False, na=False).astype(int)

  return new_df # Returns a 2D DataFrame with 7 columns

brew_transform = FunctionTransformer(brew_clean)

In [67]:
def fav_clean(x):
  # 1. Input Fix: Handle the DataFrame from the Pipeline
  if isinstance(x, pd.DataFrame):
    x = x.iloc[:, 0]

  fav_map = {
        'Blended drink (e.g. Frappuccino)': 1,
        'Mocha': 1,
        'Latte': 1,
        'Cappuccino': 1,
        'Regular drip coffee': 2,
        'Iced coffee': 2,
        'Americano': 2,
        'Cold brew': 2,
        'Other': 2,
        'Espresso': 3,
        'Cortado': 3,
        'Pourover': 4
  }

  y = x.map(fav_map).fillna(2)

  #Return as a 2D array
  return y.values.reshape(-1, 1)

fav_transform = FunctionTransformer(fav_clean)

In [68]:
def add_clean(x):
  #Ensure input is a DataFrame
  temp_df = pd.DataFrame(x)
  col_name = temp_df.columns[0]

  # List the core additions we want to track
  additions = ['Milk', 'Sugar', 'Syrup', 'Sweetener', 'Creamer']

  # 2. Match the index to keep data aligned
  new_df = pd.DataFrame(index=temp_df.index)

  for a in additions:
    new_col = f"adds_{a.lower()}"
    new_df[new_col] = temp_df[col_name].str.contains(a, case=False, na=False).astype(int)

  return new_df

add_transform = FunctionTransformer(add_clean)

In [69]:
df["prefer_overall"].unique()

array([nan, 'Coffee B', 'Coffee D', 'Coffee A', 'Coffee C'], dtype=object)

In [70]:
"""from sklearn.preprocessing import OrdinalEncoder
def age_clean(x):
  age_order=['<18 years old','18-24 years old','25-34 years old','35-44 years old','45-54 years old'
       '55-64 years old','>65 years old']
  enc = OrdinalEncoder(categories=[age_order], handle_unknown='use_encoded_value', unknown_value=-1)
  y = enc.fit_transform(x.fillna('25-34 years old'))
  return y

age_transform = FunctionTransformer(age_clean)"""

"from sklearn.preprocessing import OrdinalEncoder\ndef age_clean(x):\n  age_order=['<18 years old','18-24 years old','25-34 years old','35-44 years old','45-54 years old'\n       '55-64 years old','>65 years old']\n  enc = OrdinalEncoder(categories=[age_order], handle_unknown='use_encoded_value', unknown_value=-1)\n  y = enc.fit_transform(x.fillna('25-34 years old'))\n  return y\n\nage_transform = FunctionTransformer(age_clean)"

In [71]:
from sklearn import set_config

# This global setting forces all transformers to output DataFrames
set_config(transform_output="pandas")

In [72]:
from sklearn.preprocessing import OrdinalEncoder
pre = ColumnTransformer([
    ("age",OrdinalEncoder(categories=[['<18 years old','18-24 years old','25-34 years old','35-44 years old','45-54 years old'
       '55-64 years old','>65 years old']],handle_unknown='use_encoded_value',unknown_value=-1), ["age"]),
    ("cups", cups_transform, ["cups"]),
    ("strength", strength_transform, ["strength"]),
    ("caffeine", caffeine_transform, ["caffeine"]),
    ("expertise", expertise_transform, ["expertise"]),
    ("gender", gender_transform, ["gender"]),
    ("most_willing", willing_transform, ["most_willing"]),
    ("brew", brew_transform, ["brew"]),
    ("favorite", fav_transform, ["favorite"]),
    ("additions", add_transform, ["additions"])
]  ,remainder= "passthrough")

pre

ColumnTransformer(remainder='passthrough',
                  transformers=[('age',
                                 OrdinalEncoder(categories=[['<18 years old',
                                                             '18-24 years old',
                                                             '25-34 years old',
                                                             '35-44 years old',
                                                             '45-54 years '
                                                             'old55-64 years '
                                                             'old',
                                                             '>65 years old']],
                                                handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['age']),
                                ('cups',
                                 FunctionTransformer(func=<function cups_clean at 0x7eaee11072e0>),
                                 ['cups']),
                                ('str...
                                ('most_willing',
                                 FunctionTransformer(func=<function most_willing_clean at 0x7eaee0b20720>),
                                 ['most_willing']),
                                ('brew',
                                 FunctionTransformer(func=<function brew_clean at 0x7eaee1104720>),
                                 ['brew']),
                                ('favorite',
                                 FunctionTransformer(func=<function fav_clean at 0x7eaee1107420>),
                                 ['favorite']),
                                ('additions',
                                 FunctionTransformer(func=<function add_clean at 0x7eaee1107d80>),
                                 ['additions'])])

In [74]:
with open("coffee_pre", 'wb') as f:
    pickle.dump(pre,f)
with open("coffee_data", 'wb') as f:
    pickle.dump(data,f)
